In [1]:
from utils import Predictor, arr2db, forna_url, DataEncoder

Using TensorFlow backend.


In [2]:
# running on CPU is extremely slow!

In [3]:
import numpy as np

In [4]:
import keras

In [5]:
# model = Predictor('model/2019_09_08.1/model.hdf5')
model = Predictor('model/2019_09_12.1/model.hdf5')

W0915 14:33:12.459345 4610217408 deprecation_wrapper.py:119] From /Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0915 14:33:12.544841 4610217408 deprecation_wrapper.py:119] From /Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0915 14:33:12.546308 4610217408 deprecation_wrapper.py:119] From /Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0915 14:33:12.622814 4610217408 deprecation_wrapper.py:119] From /Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:

4


W0915 14:33:15.141752 4610217408 deprecation_wrapper.py:119] From /Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0915 14:33:15.532440 4610217408 deprecation.py:323] From /Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/tensorflow/python/ops/math_grad.py:1250: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
for l in model.model.layers:
    print l.name, l

input_org <keras.engine.input_layer.InputLayer object at 0x7fc0119a9350>
batch_normalization_1 <keras.layers.normalization.BatchNormalization object at 0x7fc0119a9550>
batch_normalization_2 <keras.layers.normalization.BatchNormalization object at 0x7fc0119a9490>
activation_1 <keras.layers.core.Activation object at 0x7fc0119a9650>
activation_2 <keras.layers.core.Activation object at 0x7fc0119a9850>
conv1d_1 <keras.layers.convolutional.Conv1D object at 0x7fc0119a9890>
conv1d_2 <keras.layers.convolutional.Conv1D object at 0x7fc0119a99d0>
batch_normalization_3 <keras.layers.normalization.BatchNormalization object at 0x7fc0119a9b10>
batch_normalization_4 <keras.layers.normalization.BatchNormalization object at 0x7fc0119a9b50>
activation_3 <keras.layers.core.Activation object at 0x7fc0119a9c90>
activation_4 <keras.layers.core.Activation object at 0x7fc0119a9dd0>
conv1d_3 <keras.layers.convolutional.Conv1D object at 0x7fc0119a9e10>
conv1d_4 <keras.layers.convolutional.Conv1D object at 0x7fc01

In [7]:
# model.model.summary(line_length=150)
model.model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_org (InputLayer)          (None, None, 4)      0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, 4)      16          input_org[0][0]                  
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, None, 4)      16          input_org[0][0]                  
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, 4)      0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
activation

In [8]:
model.model.layers[-3].name

u'lambda_9'

In [9]:
layer_input_org = next(l for l in model.model.layers if l.name == 'input_org')
layer_final_hidden = next(l for l in model.model.layers if l.name == 'conv2d_6')

layer_target_prev = next(l for l in model.model.layers if l.name == 'target_prev')
layer_concat = next(l for l in model.model.layers if l.name == 'concatenate_3')
layer_tri_conv = next(l for l in model.model.layers if l.name == 'triangular_convolution2d_1')
layer_ar_label = next(l for l in model.model.layers if l.name == 'ar_label')
layer_conv_fe = next(l for l in model.model.layers if l.name == 'conv2d_7')
layer_mask_fe = next(l for l in model.model.layers if l.name == 'lambda_9')
layer_fe = next(l for l in model.model.layers if l.name == 'fe')

In [10]:
# model1 = keras.models.Model(input=layer_input_org.input,
#                            output=layer_final_hidden.output)


In [11]:
# model1.summary()

In [12]:


model1 = keras.models.Model(input=[layer_input_org.input, layer_target_prev.input],
                            output=[layer_final_hidden.output, layer_fe.output])

new_input = keras.layers.Input(layer_final_hidden.input_shape[1:])
new_hid = layer_concat([new_input, layer_target_prev.input])
new_output = layer_tri_conv(new_hid)
new_output = layer_ar_label(new_output)
model2 = keras.models.Model(input=[new_input, layer_target_prev.input],
                            output=new_output)

/Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`
  after removing the cwd from sys.path.
/Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ar..., inputs=[<tf.Tenso...)`
  # This is added back by InteractiveShellApp.init_path()


In [13]:
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 5 0                                            
__________________________________________________________________________________________________
target_prev (InputLayer)        (None, None, None, 1 0                                            
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, None, None, 5 0           input_1[0][0]                    
                                                                 target_prev[0][0]                
__________________________________________________________________________________________________
triangular_convolution2d_1 (Tri (None, None, None, 2 82640       concatenate_3[1][0]              
__________

In [ ]:
# new_input1 = keras.layers.Input(layer_final_hidden.input_shape[1:])
# # new_input2 = keras.layers.Input(layer_target_prev.input_shape[1:])
# new_hid = layer_concat([new_input1, layer_target_prev.input])
# new_output1 = layer_tri_conv(new_hid)
# new_output1 = layer_ar_label(new_output1)
# new_output2 = layer_conv_fe(new_hid)
# new_output2 = layer_mask_fe(new_output2)  # FIXME: TF not happy with this layer =() 
# new_output2 = layer_fe(new_output2)

# model2 = keras.models.Model(input=[new_input1, new_input2],
#                             output=[new_output1, new_output2])
# # model2 = keras.models.Model(input=model.model.layers[-4].input,
# #                             output=model.model.layers[-2].output)